In [ ]:
%load_ext autoreload
%autoreload 2
from hig_data.coco2 import COCOStuffGraphPrecomputedDataset
from hig_data.utils import DataLoader
import torch
# new_coco_val2017_hig.h5
# aug_coco_train2017_hig.h5
path = '/home/rfsm2/rds/hpc-work/datasets/coco/precomputed_coco_val_paths.json'
dataset = COCOStuffGraphPrecomputedDataset(path, swapped=False)
from hig_data.utils import DataLoader
dls = DataLoader(dataset, batch_size=8, subsample=False)


In [ ]:
from training.networks_edm2_hignn_control import Precond
import torch
precond = Precond(64, 4, gnn_metadata = dataset[0].metadata()).cuda()
blank_image = torch.zeros((1, 192, 64, 64)).cuda()


In [ ]:
out, new_graph = precond.unet.hignn(blank_image, dataset[0].clone().cuda())
out.shape

In [ ]:
batch = next(iter(dls))
batch

In [ ]:
path = '/home/rfsm2/rds/hpc-work/datasets/coco/precomputed_coco_val_paths.json'
swapped_dataset = COCOStuffGraphPrecomputedDataset(path, swapped=True)

In [ ]:
i = 3
torch.allclose(dataset[3]['class_node'].x, swapped_dataset[3]['class_node'].x)

In [ ]:
dataset[0]['class_node'].x[0, :10]

In [ ]:
import h5py
path='/home/rfsm2/rds/hpc-work/datasets/coco/aug_coco_train2017_hig-sd.h5'
with h5py.File(path, 'r') as hdf:
    _data_fnames = list(hdf.keys())
    # _data_fnames.remove('statistics')
    # raw_ref_image = hdf[_data_fnames[0]]['image'][:]
    print('open', _data_fnames[0])
# print('Found {} complete datapoint in {}'.format(len(_data_fnames), path))

In [ ]:
data = dataset[2]

In [ ]:
data.bounding_boxes

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
def visualize_bounding_boxes(mask: np.ndarray,
                             bboxes: np.ndarray,
                             labels: list, bbox_color='red', text_color='white'):
    """
    Visualizes bounding boxes on a numpy array image.
    
    Arguments:
        image: 2D NumPy array representing the image.
        bboxes: Bounding boxes as an Nx4 numpy array where each row is [xmin, ymin, xmax, ymax].
                Coordinates are expected to be in 0-1 normalized format.
        bbox_color: The color for the bounding boxes (default is red).
    """
    fig, ax = plt.subplots(1)
    ax.imshow(mask,alpha=0.5)
    
    # Image dimensions
    img_h, img_w, *_ = mask.shape
    
    # Iterate over bounding boxes and draw them with labels
    for bbox, label in zip(bboxes, labels):
        # Denormalize bbox coordinates back to pixel values
        print(bbox)
        xmin, ymin, xmax, ymax = bbox * np.array([img_w, img_h, img_w, img_h])
        width = xmax - xmin
        height = ymax - ymin
        
        # Create a rectangle patch
        rect = patches.Rectangle((xmin, ymin), width, height, linewidth=1, edgecolor=bbox_color, facecolor='none')
        ax.add_patch(rect)
        
        # Add label text
        ax.text(xmin, ymin - 5, str(label), color=text_color, fontsize=8, bbox=dict(facecolor=bbox_color, alpha=0.5, pad=1))

    plt.show()
data = dataset[1011]
visualize_bounding_boxes(data.mask.squeeze(), data.bounding_boxes, data.bounding_box_class)


In [ ]:
graph_batch = next(iter(dls))

In [ ]:
from training.encoders import StabilityVAEEncoder
from hig_data.visualisation import plot_array_images
vae = StabilityVAEEncoder(batch_size=8)

sampled = vae.encode_latents(graph_batch.image.to('cuda'))
sampled_pixels = vae.decode(sampled)
sampled_pixels.shape
plot_array_images(sampled_pixels.cpu()) # rotating doesn
plot_array_images(graph_batch.mask) # rotating doesnt work


In [ ]:
%load_ext autoreload
%autoreload 2
from hig_data.coco2 import CocoStuffGraphDatasetLightweight
from hig_data.utils import DataLoader    
from training.encoders import StabilityVAEEncoder
# /home/rfsm2/rds/hpc-work/datasets/coco/coco_train_paths.json
path = '/home/rfsm2/rds/hpc-work/datasets/coco/coco_val_paths.json'
dataset = CocoStuffGraphDatasetLightweight(path)
from hig_data.utils import DataLoader  
dls = DataLoader(dataset, batch_size=8, augmentation=False)
data = next(iter(dls))

img, mask, label = data
label
# i=7
# visualize_bounding_boxes(mask[i].squeeze(), label[i]['obj_bbox'], label[i]['obj_class'])


In [ ]:
label[0]

In [ ]:
from generate_images import edm_sampler
import os
import re
import pickle
import torch

run_dir = '/home/rfsm2/rds/hpc-work/training_runs/edm2_pretrained/pretrained'
pattern=r'edm2-img512-s.*(\d+).pkl'
verbose=True
fnames = [entry.name for entry in os.scandir(run_dir) if entry.is_file() and re.fullmatch(pattern, entry.name)]
print(fnames)
pkl_path = os.path.join(run_dir, max(fnames, key=lambda x: float(re.fullmatch(pattern, x).group(1))))

if verbose:
    print(f'Loading from {pkl_path} ... ', end='', flush=True)
with open(pkl_path, 'rb') as f:
    data = pickle.load(f, fix_imports=True, encoding="bytes")

sample_shape = (8, 4, 64, 64)
noise = torch.randn(sample_shape, device='cuda')
sampled = edm_sampler(net=data.ema.to('cuda'), noise=noise, graph=None) # sample images from noise and graph batch


In [ ]:
from hig_data.visualisation import plot_array_images

vae = StabilityVAEEncoder(batch_size=8)
sampled_pixels = vae.decode(sampled)
sampled_pixels.shape
plot_array_images(sampled_pixels.cpu()) # rotating doesnt work
# plot_array_images(sampled.cpu()) # rotating doesnt work

In [ ]:
from hig_data.visualisation import plot_array_images
iter_dls = iter(dataloader)
img, mask, label = next(iter_dls)
plot_array_images(img) # rotating doesnt work

In [ ]:
img.shape

In [ ]:
dataset[1]

In [ ]:
next(iter(dls))

In [ ]:
%load_ext autoreload
%autoreload 2
from hig_data.coco import COCOStuffGraphPrecomputedDataset
import torch
path = '/home/rfsm2/rds/hpc-work/datasets/coco/coco_train2017_hig.h5'
coco_graph = COCOStuffGraphPrecomputedDataset(path)
data = coco_graph[10]
# data

In [ ]:
import json

path = '/home/rfsm2/rds/hpc-work/datasets/vg/raw_vg/attributes.json'
with open(path, 'r') as f:
    data = json.load(f) 

In [ ]:
iter_dls = iter(dls)
graph_batch = next(iter_dls)

In [ ]:
from hig_data.visualisation import logging_generate_sample_vis, visualise_het_graph_on_image_batch, plot_array_images,convert_latents_to_pixels
from training.encoders import StabilityVAEEncoder
import torchvision.transforms.functional as F

# You can now specify the interpolation mode, for example:
# InterpolationMode.BILIN

vae = StabilityVAEEncoder(precomputed_latents=True)

image = graph_batch.image
print(image.shape)
# cropped_image = image[:, :, 8:56, 8:56]
# cropped_image = torch.nn.functional.interpolate(cropped_image, size=(64,64), mode='bicubic')
# flipped_image = image.flip(-1)
# rotated_images = torch.stack([F.rotate(img, angle=5, expand=False, interpolation=F.InterpolationMode.BILINEAR) for img in image])
# cropped_image = F.center_crop(rotated_images, 32)
print(image.shape)
decoded_img = data if vae is None else convert_latents_to_pixels(image, vae)
print(decoded_img.shape)
# plot_array_images(decoded_img)


In [ ]:
plot_array_images(decoded_img) # rotating doesnt work

In [ ]:
plot_array_images(decoded_img) # flipping doesnt work

In [ ]:
plot_array_images(decoded_img) # cropping works

In [ ]:
plot_array_images(decoded_img)

In [ ]:
import numpy as np
plot_array_images(decoded_img.transpose(2,0,1)[np.newaxis,...])
plot_array_images(decoded_img2.transpose(2,0,1)[np.newaxis,...])

In [ ]:
from hig_data.coco import CocoStuffGraphDataset

img_path = '/home/rfsm2/rds/hpc-work/datasets/coco/processed_coco/coco_train2017_256-sdxl.zip'
img_path_flipped = '/home/rfsm2/rds/hpc-work/datasets/coco/processed_coco/coco_train2017_256-sdxl-f.zip'
mask_path = '/home/rfsm2/rds/hpc-work/datasets/coco/processed_coco/coco_train2017_masks_256.zip'
label_path = '/home/rfsm2/rds/hpc-work/datasets/coco/processed_coco/coco_train2017_labels.json'
caption_path = '/home/rfsm2/rds/hpc-work/datasets/coco/processed_coco/coco_train2017_caption-clip.zip'
vocab_path = "/home/rfsm2/rds/hpc-work/datasets/coco/processed_coco/class_name_clip_latents.json"
dataset = CocoStuffGraphDataset(img_path,
                                mask_path,
                                labels_path=label_path,
                                captions_path=caption_path,
                                vocab_path=vocab_path,
                                image_path_flipped=img_path_flipped,
                                xflip=True)


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dataset_tool import graphencodecoco2

graphencodecoco2(path='/home/rfsm2/rds/hpc-work/datasets/coco/coco_val_paths.json',
                 repeats=1, train=False, batch_size=100, dest='/home/rfsm2/rds/hpc-work/datasets/coco/new_coco_val2017_hig.h5')

In [ ]:

from hig_data.coco2 import CocoStuffGraphDatasetLightweight
from hig_data.utils import DataLoader
import torch
root = '/home/rfsm2/rds/hpc-work/datasets/coco/coco_val_paths.json'
dataset = CocoStuffGraphDatasetLightweight(root)

# for i in range(10):
    # img,mask,label = dataset[0]
    # print(img.shape, mask.shape)
dls = DataLoader(dataset, batch_size=8)

graph_batch = next(iter(dls))
# graph_batch
# vae = StabilityVAEEncoder(batch_size=1)

# idx = 0
# i = 0
# graph = dataset[idx]
# raw_idx = dataset._raw_idx[idx]
# image_filename = dataset._file_name(dataset._all_fnames['image'][raw_idx])
# out_fname = f'{image_filename}_{i}'
# print(graph.mask.shape)

# # convert latent
# img_tensor = graph.image.to('cuda')
# mean_std = vae.encode_pixels(img_tensor)[0].cpu()
# # store mask at latent size

# resized_mask = torch.nn.functional.interpolate(graph.mask, size=(dataset.grid_size, dataset.grid_size), mode='nearest').squeeze() # resize mask to match compression

# print(mean_std.shape, resized_mask.shape)
# from hig_data.utils import DataLoader
# dls = DataLoader(dataset, batch_size=2048)

In [ ]:
from hig_data.visualisation import plot_array_images

plot_array_images(dataset[2].image)

In [ ]:
len(dataset)
# dataset._xflip[7000]

In [ ]:
data = dataset[0]
data = dataset[236560//2]

In [ ]:
len(dataset)

In [ ]:
from hig_data.visualisation import logging_generate_sample_vis, visualise_het_graph_on_image_batch, plot_array_images,convert_latents_to_pixels
from training.encoders import StabilityVAEEncoder
vae = StabilityVAEEncoder()
data = dataset[1].image
data2 = dataset[(236560//2)+1].image


In [ ]:
import numpy as np
np.allclose(data, data2)

In [ ]:
import torch
decoded_img = data if vae is None else convert_latents_to_pixels(torch.tensor(data), vae)
decoded_img2 = data2 if vae is None else convert_latents_to_pixels(torch.tensor(data2), vae)

In [ ]:
import numpy as np
plot_array_images(decoded_img.transpose(2,0,1)[np.newaxis,...])
plot_array_images(decoded_img2.transpose(2,0,1)[np.newaxis,...])

In [ ]:
import torch
data = dataset[1]
data2 = dataset[(236560//2)+1]
bb_1 = data['instance_to_image'].edge_index
bb_2 = data2['instance_to_image'].edge_index
from hig_data.coco import linear_to_2d_idx
bb_1, bb_2
img_bb_1 = linear_to_2d_idx(torch.tensor(bb_1[1]))[:,:200]
img_bb_2 = linear_to_2d_idx(torch.tensor(bb_2[1]))[:,:200]

from hig_data.visualisation import plot_array_images
image = torch.zeros((2, 1, 32, 32))
image[0,:,img_bb_1[0], img_bb_1[1]] = 1.
image[1,:,img_bb_2[0], img_bb_2[1]] = 1.

plot_array_images(image)

In [ ]:
import torch
import numpy as np
from hig_data.visualisation import plot_array_images
image = np.zeros((2,1, 32, 32))
image[:,:,8:20, 8:20]=1.
print(np.mean(image[0]), np.mean(image[1]))
plot_array_images(image)
plot_array_images(dataset[0].image[:, :3,:,:])

In [ ]:
from torch_geometric.loader import DataLoader as GeoDataLoader
dls = GeoDataLoader(dataset, batch_size=32, shuffle=True)
graph_batch = next(iter(dls))
graph_batch


In [ ]:
precond.unet.enc['32x32_block0'].hignn

In [ ]:
graph_batch

In [ ]:
from hig_data.visualisation import logging_generate_sample_vis, visualise_het_graph_on_image_batch, plot_array_images
# logging_generate_sample_vis(graph_batch, resize_mask=True)
from training.encoders import StabilityVAEEncoder
vae = StabilityVAEEncoder()
img, decoded = visualise_het_graph_on_image_batch(graph_batch, vae=vae,)
print(img.shape)
plot_array_images(img)
plot_array_images(decoded)


In [ ]:
import torch
from training.networks_edm2_hignn import Precond
precond = Precond(32, 3, label_dim=768, gnn_metadata = data.metadata())

images = torch.randn(1, 3, 32, 32)
rnd_normal = torch.randn([images.shape[0], 1, 1, 1], device=images.device)
sigma = (rnd_normal).exp()

data = coco_graph[1]
out = precond(images, sigma, data, class_labels=data.caption)
out.shape

In [ ]:
images = torch.randn(1, 3, 32, 32)
rnd_normal = torch.randn([images.shape[0], 1, 1, 1], device=images.device)
sigma = (rnd_normal).exp()

data = coco_graph[1]
out = precond(images, sigma, data, class_labels=data.caption)
out.shape

In [ ]:
%load_ext autoreload
%autoreload 2

from hig_data.coco_annotations import CocoAnnoations

root = '/home/rfsm2/rds/hpc-work/datasets/coco/raw_coco/'
img_dir = root+'train2017'
instances_json = root+'annotations/instances_train2017.json'
stuff_json = root+'annotations/stuff_train2017.json'
captions_json = root+'annotations/captions_train2017.json'
coco_sg = CocoAnnoations(stuff_json=stuff_json, instances_json=instances_json, captions_json=captions_json)


In [ ]:
coco_sg.save_dataset_to_json(root+'coco_train2017_labels.json')

In [ ]:
from training.encoders import CLIPEncoder
clip = CLIPEncoder(batch_size=1)
text_latents = clip.encode_raw_text(text_data, device='cuda')[0].cpu().numpy().tolist()

In [ ]:
len(coco_sg.vocab['object_idx_to_name'])

In [ ]:
coco_sg.save_vocab_to_json('./coco_2017_class_names.json')

In [ ]:
coco_sg.save_dataset_to_json('/home/rfsm2/rds/hpc-work/datasets/coco/coco_val2017_labels.json')

In [ ]:
import ijson
root = '/home/rfsm2/rds/hpc-work/datasets/coco/'
instances_json = root+'annotations/instances_train2017.json'

# def load_json_stream(file_path, prefix, keys):
#     out = {}
#     with open(file_path, "rb") as f:
#         for item in ijson.items(f, 'item'):
#             out[item[keys[0]]] = []
#             for key in keys:
#                 out[item[keys[0]]].append[item[key]]
#     return out

# out_data = load_json_stream(instances_json, 'images', 'file_name')
import json
with open(instances_json) as file:
    my_dict = json.load(file)

In [ ]:
import PIL
import numpy as np
def center_crop_imagenet(image_size: int, arr: np.ndarray):
    """
    Center cropping implementation from ADM.
    https://github.com/openai/guided-diffusion/blob/8fb3ad9197f16bbc40620447b2742e13458d2831/guided_diffusion/image_datasets.py#L126
    """
    pil_image = PIL.Image.fromarray(arr)
    while min(*pil_image.size) >= 2 * image_size:
        new_size = tuple(x // 2 for x in pil_image.size)
        assert len(new_size) == 2
        pil_image = pil_image.resize(new_size, resample=PIL.Image.Resampling.BOX)

    scale = image_size / min(*pil_image.size)
    new_size = tuple(round(x * scale) for x in pil_image.size)
    assert len(new_size) == 2
    pil_image = pil_image.resize(new_size, resample=PIL.Image.Resampling.BICUBIC)

    arr = np.array(pil_image)
    crop_y = (arr.shape[0] - image_size) // 2
    crop_x = (arr.shape[1] - image_size) // 2
    return arr[crop_y: crop_y + image_size, crop_x: crop_x + image_size]
def nearest_crop(image_size: int, arr: np.ndarray):
        """
        Center cropping for semantic masks using NEAREST resampling to preserve label information.
        """
        pil_image = PIL.Image.fromarray(arr)

        # Calculate scale based on the target image size
        scale = image_size / min(*pil_image.size)
        new_size = tuple(round(x * scale) for x in pil_image.size)
        assert len(new_size) == 2

        # Resize the image using NEAREST resampling
        pil_image = pil_image.resize(new_size, resample=PIL.Image.Resampling.NEAREST)

        # Convert back to numpy array and crop the center
        arr = np.array(pil_image)
        crop_y = (arr.shape[0] - image_size) // 2
        crop_x = (arr.shape[1] - image_size) // 2
        return arr[crop_y: crop_y + image_size, crop_x: crop_x + image_size]

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
def visualize_bounding_boxes(image: np.ndarray, mask: np.ndarray, bboxes: np.ndarray, labels: list, bbox_color='red', text_color='white'):
    """
    Visualizes bounding boxes on a numpy array image.
    
    Arguments:
        image: 2D NumPy array representing the image.
        bboxes: Bounding boxes as an Nx4 numpy array where each row is [xmin, ymin, xmax, ymax].
                Coordinates are expected to be in 0-1 normalized format.
        bbox_color: The color for the bounding boxes (default is red).
    """
    fig, ax = plt.subplots(1)
    ax.imshow(image,)
    ax.imshow(mask,alpha=0.5)
    
    # Image dimensions
    img_h, img_w, *_ = image.shape
    
    # Iterate over bounding boxes and draw them with labels
    for bbox, label in zip(bboxes, labels):
        # Denormalize bbox coordinates back to pixel values
        xmin, ymin, xmax, ymax = bbox * np.array([img_w, img_h, img_w, img_h])
        width = xmax - xmin
        height = ymax - ymin
        
        # Create a rectangle patch
        rect = patches.Rectangle((xmin, ymin), width, height, linewidth=1, edgecolor=bbox_color, facecolor='none')
        ax.add_patch(rect)
        
        # Add label text
        ax.text(xmin, ymin - 5, label, color=text_color, fontsize=8, bbox=dict(facecolor=bbox_color, alpha=0.5, pad=1))

    plt.show()

In [ ]:
import PIL
import numpy as np
m=coco_sg[0]
# print(m)
img_name = m['filename']
# print(m)

img = np.array(PIL.Image.open(root+f'val2017/{img_name}.jpg'))
mask = np.array(PIL.Image.open(root+f'val2017_masks/{img_name}.png'))
img = center_crop_imagenet(256,img)
mask = nearest_crop(256,mask)

img = img.reshape(*img.shape[:2], -1)
mask = mask.reshape(*mask.shape[:2], -1)

# PIL.Image.fromarray(img)
PIL.Image.fromarray(mask[:,:,0])

# # i = 10
# # person = m['obj_bbox'][i]
print(sorted(m['obj_class_name']))
# # print(m['obj_class'])
# # print(person)

ids_present = [idx+1 for idx in np.unique(mask) if idx !=255]
print(sorted([coco_sg.vocab['object_idx_to_name'][i] for i in ids_present]))

print(set.union(set(sorted([coco_sg.vocab['object_idx_to_name'][i] for i in ids_present])), set(sorted(m['obj_class_name']))))

# # Denormalize bbox coordinates back to pixel values
# person_bb = person * np.array([256,]*4)
# person_bb = person_bb.astype(int)
# print(person_bb)
# PIL.Image.fromarray(img[person_bb[1]:person_bb[3],person_bb[0]:person_bb[2], ])
# # print(mask.shape)
# cut_out_mask = mask[person_bb[1]:person_bb[3], person_bb[0]:person_bb[2]]
# print(cut_out_mask.shape)
# PIL.Image.fromarray(mask[person_bb[1]:person_bb[3], person_bb[0]:person_bb[2],0])
# # print(mask[person_bb[1]+60:person_bb[3]-70, person_bb[0]+20:person_bb[2]-20,0])
# print(mask[person_bb[1]:person_bb[3],person_bb[0]:person_bb[2], ])

# visualize_bounding_boxes(cropped, mask, m['obj_bbox'], m['obj_class_name'])
# PIL.Image.fromarray(cropped)

In [ ]:
import json
root = '/home/rfsm2/rds/hpc-work/edm2_datasets/coco/'
instances_json = root+'annotations/instances_val2017.json'
stuff_json = root+'annotations/stuff_val2017.json'
captions_json = root+'annotations/captions_val2017.json'
person_key_points_json = root+'annotations/person_keypoints_val2017.json'

for json_file in [instances_json, stuff_json, captions_json, person_key_points_json]:
    with open(json_file, 'r') as f:
        json_file = json.load(f)
    print(list(json_file.keys()))



In [ ]:
person_key_points_json = root+'annotations/person_keypoints_val2017.json'
with open(person_key_points_json, 'r') as f:
    person_key_points = json.load(f)
list(person_key_points['annotations'][0].keys())
# person_key_points['categories']

In [ ]:
person_key_points['annotations'][0]['keypoints']

In [ ]:
stuff_json['annotations']

In [ ]:
import json
captions_json = root+'annotations/captions_val2017.json'
with open(captions_json, 'r') as f:
    captions_data = json.load(f)

print(list(captions_data.keys()))
captions_data['annotations']

In [ ]:
meta_data

In [ ]:
from hig_data.coco import COCOStuffDataset

img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
dataset = COCOStuffDataset(img_path, mask_path, 32)


In [ ]:
from hig_data.coco import COCOStuffDataset, CocoStuffGraphDataset
import torch
img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
coco_graph = CocoStuffGraphDataset(img_path, mask_path, latent_images=True)
data = coco_graph[4]
data

In [ ]:
print(data.mask)
one_hot = torch.nn.functional.one_hot(data.mask, num_classes=256)
print(one_hot.shape)
torch.argmax(one_hot[:,:,0,0,:])

In [ ]:
torch.argmax(data['class_node'].x, dim=1)

In [ ]:
data['class_node'].label

In [ ]:
from hig_data.coco import COCOStuffGraphPrecomputedDataset
import torch
path = '/Users/rupertmenneer/Downloads/coco_graph_train'
coco_graph = COCOStuffGraphPrecomputedDataset(path)
data = coco_graph[4]
data

In [ ]:
from hig_data.coco import CocoStuffGraphDataset
import numpy as np
import os
def create_graph_dataset_and_export(output_dir='/Users/rupertmenneer/Downloads/coco_graph'):

    img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
    mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
    coco_graph = CocoStuffGraphDataset(img_path, mask_path, latent_images=True)

    for idx in range(len(coco_graph)):
        image, mask = coco_graph.dataset[idx]
        graph = coco_graph[idx]

        name = coco_graph.dataset._fnames['image'][coco_graph.dataset._raw_idx[idx]]
        raw_id = os.path.splitext(os.path.basename(name))[0].split('-')[-1]

        np.save(os.path.join(output_dir, f'{raw_id}_image.npy'), image)
        np.save(os.path.join(output_dir, f'{raw_id}_mask.npy'), mask)
        np.savez(os.path.join(output_dir, f'{raw_id}_graph'),
                 class_node=graph['class_node'].x,
                 class_pos=graph['class_node'].pos,
                 class_edge=graph['class_node', 'class_edge', 'class_node'].edge_index,
                 class_to_image=graph['class_node', 'class_to_image', 'image_node'].edge_index,
                 )

create_graph_dataset_and_export()


In [ ]:
# from hig_data.coco import COCOStuffDataset, CocoStuffGraphDataset
# import torch
# img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
# mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
# coco_graph = CocoStuffGraphDataset(img_path, mask_path, latent_images=True)
# data = coco_graph[4]
# data

In [ ]:
128<<13

In [ ]:
# from hig_data.visualisation import convert_latents_to_pixels
# import PIL
# import numpy as np
# print(data.image.shape)
# img = convert_latents_to_pixels(data.image)

# PIL.Image.fromarray(img)

In [ ]:
from torch_geometric.loader import DataLoader as GeoDataLoader
dls = GeoDataLoader(coco_graph, batch_size=32, shuffle=True)
# graph_batch = next(iter(dls))
# graph_batch

# graph_batch['image_node'].pos, graph_batch['class_node'].pos

In [ ]:
graph_batch = next(iter(dls))
graph_batch

In [ ]:
graph_batch['class_node'].label[5]

In [ ]:
for i, graph_batch in enumerate(dls):
    print(i, graph_batch.image.shape)

In [ ]:
graph_batch.detach()

In [ ]:
graph_batch['class_node'].label

In [ ]:
torch.zeros((2, 0), dtype=torch.float32)

In [ ]:
graph_batch['image_node'].pos, graph_batch['class_node'].pos

In [ ]:
# from hig_data.visualisation import save_image_batch_list
# print(graph_batch.image.shape)
# np_img = graph_batch.image[:, :3].cpu().numpy().transpose(0, 2, 3, 1)
# save_image_batch_list([np_img,np_img], row_labels=['img', 'img'], sample_batch_size=8)

In [ ]:
from hig_data.visualisation import logging_generate_sample_vis, visualise_het_graph_on_image_batch, plot_array_images
# logging_generate_sample_vis(graph_batch, resize_mask=True)
from training.encoders import StabilityVAEEncoder
vae = StabilityVAEEncoder()
img, decoded = visualise_het_graph_on_image_batch(graph_batch, vae=vae,)
print(img.shape)
plot_array_images(img)


In [ ]:
plot_array_images(decoded)

In [ ]:
import torch
from training.networks_edm2_hignn import Precond
precond = Precond(32, 3, gnn_metadata = data.metadata())

images = torch.randn(1, 3, 32, 32)
rnd_normal = torch.randn([images.shape[0], 1, 1, 1], device=images.device)
sigma = (rnd_normal).exp()

data = coco_graph[1]
out = precond(images, sigma, data)
out.shape